In [3]:
#NECESSARIO
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.1',image_scope='inference',instance_type='ml.c5.4xlarge')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[11/22/24 12:35:13] INFO     Found credentials from IAM Role:                                   ]8;id=989185;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=347532;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu'

In [ ]:
!docker pull 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.1-cpu-py310

In [ ]:
!tar -xzf model.tar.gz

In [9]:
#NECESSARIO
!pip install tensorflow keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 26.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 18.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 27.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 19.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 17.2 MB/s eta 0:00:00:00:01


In [ ]:
#NECESSARIO
!pip freeze

In [ ]:
# OPZIONALE

import os
#os.chdir('Esame')  # Sposta la working directory nella cartella "Esame"
print("Nuova directory corrente:", os.getcwd())


import os
print("Files in current directory:", os.listdir())
print("Current working directory:", os.getcwd())
# Installa le librerie specificate in requirements_inference.txt
!pip install -r requirements_inference.txt

In [48]:
#NECESSARIO SE NON SI HA IL MODELLO E IL FILE .PKL, ALTRIMENTI OPZIONALE
import os
import boto3
import zipfile

# Configura i dettagli di S3
bucket_name = 'itsar123-bianchini'  # Nome del tuo bucket S3
s3_zip_path = 'data/input/train.zip'  # Percorso del file .zip su S3
local_zip_path = 'train.zip'  # Percorso locale dove salvare il file .zip
local_path = 'data/input'  # Directory locale per estrarre i dati
s3_prefix = 'data/input'  # Percorso su S3 dove caricare la cartella estratta (opzionale)

# Step 1: Scaricare il file zip da S3
s3 = boto3.client('s3')
s3.download_file(bucket_name, s3_zip_path, local_zip_path)
print(f"File {s3_zip_path} scaricato da S3 e salvato come {local_zip_path}")

# Step 2: Decomprimere il file zip
os.makedirs(local_path, exist_ok=True)  # Crea la directory se non esiste
with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
    zip_ref.extractall(local_path)
print(f"File zip estratto nella directory: {local_path}")

#Step 3: (Opzionale) Caricare la cartella estratta su S3
for root, dirs, files in os.walk(local_path):
    for file in files:
        local_file_path = os.path.join(root, file)
        s3_file_path = os.path.relpath(local_file_path, local_path)
        s3.upload_file(local_file_path, bucket_name, os.path.join(s3_prefix, s3_file_path))
print("Cartella estratta caricata su S3.")


File data/input/train.zip scaricato da S3 e salvato come train.zip
File zip estratto nella directory: data/input
Cartella estratta caricata su S3.


In [ ]:
#NECESSARIO SE NON SI HA IL MODELLO E IL FILE .PKL, ALTRIMENTI OPZIONALE

# Configurazione della sessione AWS
my_session = boto3.session.Session()
aws_region = my_session.region_name

# Client SageMaker
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)
sagemaker_role = get_execution_role()

# Definizione di alcuni parametri
bucket_name = 'itsar123-bianchini'  # Sostituisci con il nome del tuo bucket S3
s3_prefix = 'train'  # Sostituisci con il percorso della cartella 'train' su S
data_dir = 'data'


In [ ]:
#NECESSARIO SE NON SI HA IL MODELLO E IL FILE .PKL, ALTRIMENTI OPZIONALE

# Imposta i generatori di dati
train_dir = os.path.join(data_dir, s3_prefix)
batch_size = 32
image_size = (224, 224)

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Salva la mappatura delle classi
class_indices = train_generator.class_indices
with open('class_indices.pkl', 'wb') as f:
    pickle.dump(class_indices, f)


In [ ]:
#NECESSARIO SE NON SI HA IL MODELLO E IL FILE .PKL, ALTRIMENTI OPZIONALE

# Costruisci il modello (puoi personalizzarlo o utilizzare un modello pre-addestrato)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

# Compila il modello
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Allena il modello
epochs = 4  # Puoi aumentare il numero di epoche se necessario
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Salva il modello
model.save('card_model.h5')


In [ ]:
#NECESSARIO SE NON SI HA IL MODELLO E IL FILE .PKL, ALTRIMENTI OPZIONALE

# Passo 2: Salva il modello nel formato .keras
model.save('model.keras', save_format='keras')
print("Il modello è stato salvato come .keras.")

In [ ]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

In [6]:
#STARTING POINT POST IMPORT LIBRERIE
from keras import models
model = models.load_model("model.keras")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 #STARTING POINT POST IMPORT LIBRERIE                                                         │
│ ❱ 2 from keras import models                                                                     │
│   3 model = models.load_model("model.keras")                                                     │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'keras'

In [7]:
import pickle

with open('model/class_indices.pkl', 'rb') as f:
    class_indices = pickle.load(f)

print("Class Indices:", class_indices)


Class Indices: {'ace of clubs': 0, 'ace of diamonds': 1, 'ace of hearts': 2, 'ace of spades': 3, 'eight of clubs': 4, 'eight of diamonds': 5, 'eight of hearts': 6, 'eight of spades': 7, 'five of clubs': 8, 'five of diamonds': 9, 'five of hearts': 10, 'five of spades': 11, 'four of clubs': 12, 'four of diamonds': 13, 'four of hearts': 14, 'four of spades': 15, 'jack of clubs': 16, 'jack of diamonds': 17, 'jack of hearts': 18, 'jack of spades': 19, 'joker': 20, 'king of clubs': 21, 'king of diamonds': 22, 'king of hearts': 23, 'king of spades': 24, 'nine of clubs': 25, 'nine of diamonds': 26, 'nine of hearts': 27, 'nine of spades': 28, 'queen of clubs': 29, 'queen of diamonds': 30, 'queen of hearts': 31, 'queen of spades': 32, 'seven of clubs': 33, 'seven of diamonds': 34, 'seven of hearts': 35, 'seven of spades': 36, 'six of clubs': 37, 'six of diamonds': 38, 'six of hearts': 39, 'six of spades': 40, 'ten of clubs': 41, 'ten of diamonds': 42, 'ten of hearts': 43, 'ten of spades': 44, 't

In [11]:
from keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import numpy as np
import pickle
import json

# Carica il modello
model = load_model("model/model.keras")

# Carica la mappatura delle classi
with open('model/class_indices.pkl', 'rb') as f:
    class_indices = pickle.load(f)
# Creazione di una mappatura inversa
class_labels = {v: k for k, v in class_indices.items()}

# Creazione del preprocessore ImageDataGenerator
image_size = (224, 224)  # Dimensione target utilizzata durante l'addestramento
datagen = ImageDataGenerator(rescale=1./255)

# Funzione per preprocessare una singola immagine
def preprocess_single_image(image_path, datagen, target_size):
    img = load_img(image_path, target_size=target_size)  # Carica e ridimensiona l'immagine
    img_array = img_to_array(img)  # Converte in array
    img_array = np.expand_dims(img_array, axis=0)  # Aggiunge una dimensione batch
    img_array = datagen.standardize(img_array)  # Applica la stessa normalizzazione
    return img_array

# Percorso all'immagine
# image_path = "Asso_Di_Cuori.jpg"
# image_path = "Regina_Di_Picche.jpg"
image_path = "Quattro_Di_Quadri.jpg"

# Preprocessa l'immagine
predict_input = preprocess_single_image(image_path, datagen, image_size)

# Esegui la previsione
predict_result = model.predict(predict_input)

# Estrai la classe predetta
predicted_class_index = np.argmax(predict_result, axis=1)[0]  # Indice della classe predetta
predicted_class_label = class_labels[predicted_class_index]  # Nome della classe

# Supponendo che la label segua il formato "valore of seme"
if " of " in predicted_class_label:
    valore, seme = predicted_class_label.split(" of ")  # Separa valore e seme
else:
    valore = predicted_class_label  # L'intera label è considerata il valore
    seme = "Seme non specificato"

# Converti in JSON il risultato
output = json.dumps({
    "valore": valore,
    "seme": seme,
    #"predict_result": predict_result.tolist()
})
print(output)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
{"valore": "four", "seme": "diamonds"}


In [12]:
!pip install flask

In [67]:
%%sh
# Specify an image name
image_name=tensorflow-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-inference ######################
account: 533267394241 ######################
region: us-east-1 ######################
fullname: 533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.0-cpu-py310-pluto-2 ######################
{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:533267394241:repository/tensorflow-inference",
            "registryId": "533267394241",
            "repositoryName": "tensorflow-inference",
            "repositoryUri": "533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference",
            "createdAt": 1732280998.63,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        }
    ]
}


In [68]:
#Dominio dell'immagine base, quindi è uguale per tutti
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [94]:
#Codice build è univoco per persona
!docker build -t 533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo -f Dockerfile.inference .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 501B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.2s (10/11)                                        docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 501B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [93]:
!docker images

REPOSITORY                                                          TAG       IMAGE ID       CREATED             SIZE
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   mondo     a54d775edf29   5 seconds ago       4.55GB
<none>                                                              <none>    65c1da710c89   4 minutes ago       5.08GB
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   <none>    3a6f714725a6   48 minutes ago      5.08GB
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   <none>    e5d7bbc0de2e   About an hour ago   5.08GB
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   carte     b53715e0cf64   About an hour ago   5.08GB
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training    latest    fff49fa6705a   2 hours ago         8.59GB
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   <none>    63a085322590   2 hours ago         5.08GB
<none>                                    

In [71]:
#Dominio dell'immagine base, quindi è uguale per tutti
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 533267394241.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [95]:
!docker push 533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo

The push refers to repository [533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference]

98481e58: Preparing 
077cb0ae: Preparing 
7555843b: Preparing 
64520764: Preparing 
37bf1d61: Preparing 
8df97c44: Preparing 
764747b4: Preparing 
1a68f579: Preparing 
2eef5eea: Preparing 
b5bcc575: Preparing 
5e5ce62f: Preparing 
757a305d: Preparing 
c7cf6f28: Preparing 
30092134: Preparing 
fb153852: Preparing 
0903db8c: Preparing 
19dec72a: Preparing 
df04f233: Preparing 
f2dbc490: Preparing 
ba0431f9: Preparing 
71536788: Preparing 
994107ae: Preparing 
3a4f83e7: Preparing 
d6748243: Preparing 
f2c1e372: Preparing 
cd2b5d6d: Preparing 
8481e58: Pushed lready exists 2kB2Amondo: digest: sha256:2b6e6eede9097f96f311dcaf9c251d2b0c477cac599b4201fed3a25a2b2eee86 size: 5973


In [39]:
%%sh
# Specify an image name
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-training ######################
account: 533267394241 ######################
region: us-east-1 ######################
fullname: 533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-pluto-2 ######################



An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'tensorflow-training' does not exist in the registry with id '533267394241'


{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:533267394241:repository/tensorflow-training",
        "registryId": "533267394241",
        "repositoryName": "tensorflow-training",
        "repositoryUri": "533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training",
        "createdAt": 1732282970.746,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [40]:
#Dominio dell'immagine base, quindi è uguale per tutti
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 533267394241.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [ ]:
#Codice build è univoco per persona
!docker build -t 533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training -f Dockerfile.train .

In [84]:
!docker images

REPOSITORY                                                          TAG       IMAGE ID       CREATED          SIZE
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   mondo     3a6f714725a6   35 minutes ago   5.08GB
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   <none>    e5d7bbc0de2e   47 minutes ago   5.08GB
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   carte     b53715e0cf64   47 minutes ago   5.08GB
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training    latest    fff49fa6705a   2 hours ago      8.59GB
533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference   <none>    63a085322590   2 hours ago      5.08GB
<none>                                                              <none>    7238b6a86d0e   2 hours ago      5.08GB


In [43]:
!docker push 533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:latest

The push refers to repository [533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training]

2bb916d7: Preparing 
6cf7bbf6: Preparing 
22878ecf: Preparing 
db4f6696: Preparing 
f529ae49: Preparing 
75f13e59: Preparing 
0c6c24f9: Preparing 
a1e64371: Preparing 
29631b13: Preparing 
3f03efd0: Preparing 
9b75d5ee: Preparing 
370de786: Preparing 
a16f1a7a: Preparing 
fc8740d1: Preparing 
292691fc: Preparing 
6672bf9d: Preparing 
607be3da: Preparing 
afc9b360: Preparing 
f934e4df: Preparing 
80a2a6eb: Preparing 
a0538fa4: Preparing 
6d109c54: Preparing 
f0a068df: Preparing 
443b7673: Preparing 
37a36861: Preparing 
ecb4bd81: Preparing 
34b60c81: Preparing 
f03efd0: Waiting g 
b75d5ee: Waiting g 
70de786: Waiting g 
16f1a7a: Waiting g 
ab822c1c: Pushed   755.5MB/731.1MBPushing  144.2MB/443.1MBPushing  554.8MB/1.042GBPushing  752.6MB/2.455GBPushing  395.7MB/502.1MBPushing  457.8MB/502.1MBPushing  124.1MB/731.1MBPushing  2.111GB/2.217GBPushing  636.1MB/731.1MBPushing  2.146GB/2.217GBlates

In [ ]:
#OPZIONALE, SE SI VUOLE ALLENARE IL MODELLO CON I DATI SU S3

from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os
import pandas as pd

role=get_execution_role()

MODEL_INPUT_PATH = 's3://itsar123-bianchini/data/input/train'

MODEL_OUTPUT_PATH = 's3://itsar123-bianchini/data/output'

INSTANCE_TYPE = 'ml.p2.xlarge'

hyperparameters={'epochs': 10}

estimator=Estimator(
    image_uri='915906710248.dkr.ecr.us-east-1.amazonaws.com/my-tensorflow-training:latest,
    role=role,
    instance_count=1,
    instance_type=INSTANCE_TYPE,
    hyperparameters=hyperparameters,
    output_path=MODEL_OUTPUT_PATH
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit(MODEL_INPUT_PATH)
print('##### ESTIMATOR FIT COMPLETED')

In [47]:
#OPZIONALE, SE SI VUOLE ARCHIVIARE IL MODELLO

import tarfile
import os

# Percorsi dei file da includere
model_file = "model/model.keras"
class_indices_file = "model/class_indices.pkl"
inference_file = "inference.py"

# Nome del file tar.gz di output
output_tar_gz = "card_model.tar.gz"

# Creazione dell'archivio tar.gz
with tarfile.open(output_tar_gz, "w:gz") as tar:
    # Aggiungi il file del modello
    tar.add(model_file, arcname=os.path.basename(model_file))
    # Aggiungi il file class_indices.pkl
    tar.add(class_indices_file, arcname=os.path.basename(class_indices_file))
    # Aggiungi il file inference.py
    tar.add(inference_file, arcname=os.path.basename(inference_file))

print(f"{output_tar_gz} creato con successo!")

card_model.tar.gz creato con successo!


In [87]:
!tar -czf model.tar.gz model/model.keras model/class_indices.pkl

In [88]:
!ls -lha model.*

-rw-rw-r-- 1 ec2-user ec2-user 444M Nov 22 15:28 model.tar.gz


In [89]:
!aws s3 cp ./model.tar.gz s3://itsar123-bianchini/data/output/model.tar.gz

upload: ./model.tar.gz to s3://itsar123-bianchini/data/output/model.tar.gz


In [91]:
!chmod +x pippo.sh

In [97]:
from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

my_session = boto3.session.Session()
aws_region = my_session.region_name

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

sagemaker_role = get_execution_role()

model_name = 'modello-carte'

# Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': '533267394241.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo',
        'ModelDataUrl': 's3://itsar123-bianchini/data/output/model.tar.gz',
    })

In [100]:
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'la-mia-quinta-api-inferenza-config'

instance_type = 'ml.m5.large' 

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name,
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)
print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = 'la-mia-quinta-api-inferenza'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)

Created EndpointConfig: arn:aws:sagemaker:us-east-1:533267394241:endpoint-config/la-mia-quinta-api-inferenza-config


In [54]:
!aws sagemaker describe-endpoint --endpoint-name la-mia-prima-api-inferenza --region us-east-1

{
    "EndpointName": "la-mia-prima-api-inferenza",
    "EndpointArn": "arn:aws:sagemaker:us-east-1:533267394241:endpoint/la-mia-prima-api-inferenza",
    "EndpointConfigName": "la-mia-prima-api-inferenza-config",
    "EndpointStatus": "Creating",
    "CreationTime": 1732284214.275,
    "LastModifiedTime": 1732284214.822
}


In [57]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name la-mia-prima-api-inferenza --region us-east-1

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: the following arguments are required: --body, outfile


In [74]:
# Carica il modello e la mappatura delle classi
model_dir = os.environ.get('SM_MODEL_DIR', '/opt/ml/model')
print(f"######## La model dir è: {model_dir}")

######## La model dir è: /opt/ml/model


In [ ]:
%%sh
aws logs get-log-events \
    --log-group-name /aws/sagemaker/Endpoints/la-mia-terza-api-inferenza \
    --log-stream-name <nome-stream-log> \
    --region us-east-1
